# Flow
- User-based cosine similarity를 통한 10명의 유사한 유저 선출
- 해당 유저들이 작성한 리뷰데이터를 가져와 TF - IDF와 Sentiment Polarity 계산
- 최종 Top 3의 추천 레스토랑 선정

![flow](./recommender_flow.png)

In [1]:
import pandas as pd

In [2]:
# main으로 사용할 데이터프레임 (리뷰데이터)
main = pd.read_csv("./main_sliced.csv", index_col=0)
main

C:\Users\PIAI\AppData\Local\Temp\ipykernel_2376\84168529.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  main = pd.read_csv("./main_sliced.csv", index_col=0)


,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,RestaurantsDelivery,RestaurantsTableService,Monday,Tuesday,Wednesday,Thursday,Friday,...,funny,cool,text,date,year,time,month,day,text_clear,senti_polarity
1,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,1,2,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,2013,15:19:06,10,5,bakery usually go chinatown decent variety bun...,0.094872
4,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,1,1,HK-STYLE MILK TEA: FOUR STARS Not quite sure w...,2013-10-25 02:31:35,2013,02:31:35,10,25,hkstyle milk tea four stars quite sure two sai...,0.162063
5,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,0,0,This is my favorite bakery in Chinatown! I usu...,2018-03-17 23:47:25,2018,23:47:25,3,17,favorite bakery chinatown usually get pineappl...,0.173333
6,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,0,0,so I dropped in to this store since it wa the ...,2012-08-16 19:42:17,2012,19:42:17,8,16,dropped store since wa company would find hong...,0.272500
7,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,4,10,Impression: The good thing is that there are l...,2017-04-20 12:37:09,2017,12:37:09,4,20,impression good thing like kind asian chinese ...,0.109432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185501,True,2.0,True,True,True,11:0-22:0,11:0-22:0,NaN,11:0-22:0,11:0-2:0,...,0,1,Would have been 5 star if not for the single c...,2019-03-18 06:24:20,2019,06:24:20,3,18,would star single carnitas taco nasty amount f...,-0.050794
1185503,True,2.0,True,True,True,11:0-22:0,11:0-22:0,NaN,11:0-22:0,11:0-2:0,...,4,1,After Blue Corn refused to seat me and my wife...,2019-10-06 09:05:35,2019,09:05:35,10,6,blue corn refused seat wifes family stroller u...,0.191991
1185506,True,2.0,True,True,True,11:0-22:0,11:0-22:0,NaN,11:0-22:0,11:0-2:0,...,1,1,First time trying this restaurant and I had a ...,2020-11-30 23:19:40,2020,23:19:40,11,30,first time trying restaurant good experience o...,0.358897
1185508,True,2.0,True,True,True,11:0-22:0,11:0-22:0,NaN,11:0-22:0,11:0-2:0,...,0,0,Recently got take out from adelita; they were ...,2021-02-07 15:09:25,2021,15:09:25,2,7,recently got take adelita great super fast ord...,0.513333


In [3]:
# item - user matrix를 계산할 때, 동일 유저가 동일 레스토랑에 여러번 방문하여 리뷰를 남길경우 matrix 생성시 데이터가 평균이 아닌 합산으로 들어가게됨.
# 이를 방지하기위해 미리 중복되는 데이터에 대해 평균을 계산하여 새로 데이터프레임으로 저장. 
item_user = pd.read_csv('./user_item_df.csv', index_col=0)
item_user

C:\Users\PIAI\AppData\Local\Temp\ipykernel_2376\2038725877.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  item_user = pd.read_csv('./user_item_df.csv', index_col=0)


,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,RestaurantsDelivery,RestaurantsTableService,Monday,Tuesday,Wednesday,Thursday,Friday,...,funny,cool,text,date,year,time,month,day,text_clear,senti_polarity
1,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,1,2,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,2013,15:19:06,10,5,bakery usually go chinatown decent variety bun...,0.094872
5,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,0,0,This is my favorite bakery in Chinatown! I usu...,2018-03-17 23:47:25,2018,23:47:25,3,17,favorite bakery chinatown usually get pineappl...,0.173333
6,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,0,0,so I dropped in to this store since it wa the ...,2012-08-16 19:42:17,2012,19:42:17,8,16,dropped store since wa company would find hong...,0.272500
7,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,4,10,Impression: The good thing is that there are l...,2017-04-20 12:37:09,2017,12:37:09,4,20,impression good thing like kind asian chinese ...,0.109432
8,False,1.0,True,False,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,...,0,0,Best egg tarts? I wa lucky to get them fresh a...,2014-02-25 14:41:08,2014,14:41:08,2,25,best egg tarts wa lucky get fresh right oven o...,0.474226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847169,False,1.0,True,False,NaN,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,...,0,0,Just an update after Tom stopped by Broad and ...,2010-09-01 15:07:31,2010,15:07:31,9,1,update tom stopped broad lombard breakfast day...,0.307000
969141,True,1.0,True,True,NaN,0:0-0:0,NaN,NaN,8:0-18:0,8:0-18:0,...,0,0,Flying Monkey ha taken a great leap forward wi...,2010-11-04 17:55:47,2010,17:55:47,11,4,flying monkey ha taken great leap forward new ...,0.052683
913843,True,1.0,True,True,False,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,...,3,2,If you think your not cool enough to go in thi...,2016-12-19 02:22:07,2016,02:22:07,12,19,think cool enough go place youre probably righ...,-0.028571
1039342,True,1.0,True,True,NaN,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-0:0,...,0,0,"i use to love this place, but now night time i...",2011-03-11 19:23:26,2011,19:23:26,3,11,use love place night time time go sam man last...,0.260000


# Input

In [4]:
# 임의로 Input 지정.
# 지정 카테고리의 경우 너무 포괄적이지 않음과 동시에 어느정도의 리뷰 수가 보장되는 경우 성능이 더 좋게 나타남. 
recomm_category = ' coffee & tea'
recomm_user = 'zzBhA0M7NNBMYoWj48h53A'

In [5]:
# df의 카테고리 (1~10)에 해당하는 카테고리 인덱스가 있는 데이터만 다시 가져오기 
target_cat_df = main[main['Cat1'].str.contains(recomm_category) | main['Cat2'].str.contains(recomm_category) | main['Cat3'].str.contains(recomm_category) | main['Cat4'].str.contains(recomm_category)| main['Cat5'].str.contains(recomm_category)|main['Cat6'].str.contains(recomm_category)|main['Cat7'].str.contains(recomm_category)|main['Cat8'].str.contains(recomm_category)|main['Cat9'].str.contains(recomm_category)|main['Cat10'].str.contains(recomm_category)]
target_cat_df.shape

(34487, 45)

# Phase 1 : User Similarity

In [6]:
item_user_df =  item_user[item_user['Cat1'].str.contains(recomm_category) | item_user['Cat2'].str.contains(recomm_category) | item_user['Cat3'].str.contains(recomm_category) | item_user['Cat4'].str.contains(recomm_category)| item_user['Cat5'].str.contains(recomm_category)|item_user['Cat6'].str.contains(recomm_category)|item_user['Cat7'].str.contains(recomm_category)|item_user['Cat8'].str.contains(recomm_category)|item_user['Cat9'].str.contains(recomm_category)|item_user['Cat10'].str.contains(recomm_category)]
item_user_df.shape

(30852, 45)

In [7]:
import numpy as np
#file=pd.read_csv("data.csv",names=['user','item','rating','timestamp'])

from scipy.sparse import csr_matrix

user_u = list(sorted(item_user_df.user_id.unique()))
item_u = list(sorted(item_user_df.name.unique()))

# cat.codes = 카테고리를 수치형으로 변환하는 get dummies와 같은 느낌
row = item_user_df.user_id.astype('category').cat.codes
col = item_user_df.name.astype('category').cat.codes

data = item_user_df['stars_y'].tolist()

sparse_matrix = csr_matrix((data, (row, col)), shape=(len(user_u), len(item_u)))

# 그냥 pivot table을 만들려고 하면 sparse하기 때문에 negative dimension은 허용되지않는다는 오류가 뜬다. 
# SparseDataFrame으로 진행하려 했지만 더이상 pandas에서 지원되지 않음 -> sparseArray로 진행 
user_item_matrix = pd.DataFrame([pd.arrays.SparseArray(sparse_matrix[i].toarray().ravel(), fill_value=0) 
                              for i in np.arange(sparse_matrix.shape[0]) ], 
                       index=user_u, columns=item_u).fillna(0)

In [8]:
user_item_matrix.shape

(7792, 735)

In [9]:
# 기존에 sum문제로 평점의 범위가 0 ~ 68이였던 것에 비해 모두 0 ~5점 사이의 값으로 계산 되었음을 알 수 있음.
user_item_matrix.loc['HWQNlFjchIN37v-aPJEQDQ'].value_counts()

0    727
3      4
4      2
5      2
Name: HWQNlFjchIN37v-aPJEQDQ, dtype: int64

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(user_item_matrix,user_item_matrix)

In [11]:
user_sim.shape

(7792, 7792)

In [12]:
user_sim_df = pd.DataFrame(data = user_sim, index = user_item_matrix.index, columns = user_item_matrix.index)
user_sim_df.head()

,-1MF2tosrw2WcCxeVNk81Q,-1W0vSD9aJXnHL63xiEciw,-2cKJFFNJ9XVyWBt62mWvA,-3h48rG-sopyGDLu5bh4xQ,-4RbxLJlFZlu-KRuUiiGLw,-4qa2JVDEC_QPVWcZ1nHkQ,-5Sp9MylB4jv52boC4c3wg,-6GY04bTPM2Zo4z0GN4a1A,-6jjIGPGRgKUwLkz5-5UlA,-7-d8x5w5c9QolAMgPTKZg,...,zt0FvOuRJSu_olRZixNLAw,ztDSO7DSGNtPN_XW5VFjvg,zu-e06_BM_TdkAZEKMrIww,zurt07xTUcGv1wzFipjXBQ,zvHe_sQhUk6vxE_svqh9DA,zvK7PGyyvWMLqpdX6LCZTg,zvq6qxc2IVA08jaUelUMRw,zyNrXvJyYdC34tS6BcCykA,zzBhA0M7NNBMYoWj48h53A,zzeRWIiPtuJNRBUcxe0Upw
-1MF2tosrw2WcCxeVNk81Q,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
-1W0vSD9aJXnHL63xiEciw,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
-2cKJFFNJ9XVyWBt62mWvA,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
-3h48rG-sopyGDLu5bh4xQ,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
-4RbxLJlFZlu-KRuUiiGLw,0.0,0.0,0.0,0.0,1.0,0.095265,0.441726,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.223105,0.0,0.0,0.0


In [13]:
# user similarity top 10을 가져오고 저장하는 코드
top10_list= user_sim_df[recomm_user].sort_values(ascending=False)[1:11]

sim = []
for s in top10_list:
    sim.append(s)
    
top10_df = pd.DataFrame({'user_id':top10_list.index, 'sim':sim})

In [14]:
top10_df

,user_id,sim
0,nW5NEfaNwcw1vGnCRfvYXA,0.928477
1,Qhwzp9RmonEhIPRqjxD2eA,0.928477
2,hVUcXc-qwDlj_ihiD6Xuyg,0.928477
3,Zywg_JbfMjlGI0tWVhqNKw,0.830455
4,bAtvLtIOCwt72zFXiBDYlg,0.804639
5,j7gM2jTyQu3EIXVDvZSQ8Q,0.707957
6,Z6zAW5KYUbOqlv1pa63STw,0.704248
7,etUipbnxRboLh00gz78yXA,0.702069
8,LFhKGPTKgGYA0izUtDdraA,0.664364
9,VhHlJRcOg5BgXq9FYg73BQ,0.656532


In [15]:
# 유사한 유저 아이디만 가져오기
sim_users = top10_df['user_id']
sim_users

0    nW5NEfaNwcw1vGnCRfvYXA
1    Qhwzp9RmonEhIPRqjxD2eA
2    hVUcXc-qwDlj_ihiD6Xuyg
3    Zywg_JbfMjlGI0tWVhqNKw
4    bAtvLtIOCwt72zFXiBDYlg
5    j7gM2jTyQu3EIXVDvZSQ8Q
6    Z6zAW5KYUbOqlv1pa63STw
7    etUipbnxRboLh00gz78yXA
8    LFhKGPTKgGYA0izUtDdraA
9    VhHlJRcOg5BgXq9FYg73BQ
Name: user_id, dtype: object

In [120]:
# user sim top10에서 별점이 5점인 레스토랑에 대한 정보만 가져오기
phase1_list = set()
for s in sim_users:
    _df = item_user[item_user['user_id']== s][['stars_y','name']]
    for d in _df.iterrows():
        if d[1][0] == 5: # 별점이 5점인 경우
            phase1_list.add(d[1][1]) # 해당 레스토랑 이름 가져오기 (중복되지 않도록 set으로 저장)
phase1_list = list(phase1_list)

In [123]:
phase1_df = item_user[item_user['name'].isin(phase1_list)][['name','stars_y']].sort_values('stars_y', ascending=False)

In [124]:
phase1_df

,name,stars_y
1122397,reading terminal market,5
766510,lucha cartel,5
766583,lucha cartel,5
1121646,reading terminal market,5
1121647,reading terminal market,5
...,...,...
766224,lucha cartel,1
33421,bareburger - midtown village,1
33258,bareburger - midtown village,1
33736,bareburger - midtown village,1


In [125]:
phase1_df = phase1_df.drop_duplicates(['name'], keep='first')
phase1_df

,name,stars_y
1122397,reading terminal market,5
766510,lucha cartel,5
833596,profi's crêperie,5
957131,barra rossa ristorante,5
750882,triangle tavern,5
740861,the m room,5
737484,george's market at dreshertown,5
581553,wrap shack,5
999227,black & brew,5
1008994,federal beer distributor,5


In [126]:
phase1_df = phase1_df.sort_values('stars_y', ascending=False)

In [127]:
phase1_df.name.value_counts() # 중복없이 들어감 체크 

reading terminal market            1
lucha cartel                       1
profi's crêperie                   1
barra rossa ristorante             1
triangle tavern                    1
the m room                         1
george's market at dreshertown     1
wrap shack                         1
black & brew                       1
federal beer distributor           1
harp & crown                       1
essen bakery                       1
bareburger - midtown village       1
the sweet taco                     1
blue cross riverrink winterfest    1
Name: name, dtype: int64

# Phase 2 ; TF - IDF Similarity

In [23]:
# 유사도 높은 10개의 유저에 대한 df(리뷰들)만 가져옴 (해당 카테고리에 대해)
second_target_df = target_cat_df[target_cat_df['user_id'].isin(sim_users)]
second_target_df.shape

(29, 45)

In [24]:
second_target_df

,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,RestaurantsDelivery,RestaurantsTableService,Monday,Tuesday,Wednesday,Thursday,Friday,...,funny,cool,text,date,year,time,month,day,text_clear,senti_polarity
8247,True,1.0,True,False,NaN,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0,...,0,0,I have been going to this Cosi with terrible r...,2012-01-07 21:42:45,2012,21:42:45,1,7,going cosi terrible result since wa teenager k...,0.038127
29101,True,2.0,True,False,NaN,NaN,11:30-0:0,11:30-0:0,11:30-2:0,11:30-2:0,...,0,2,"This weekend, I made my first trip to Norristo...",2014-10-13 13:53:01,2014,13:53:01,10,13,weekend made first trip norristown year lived ...,0.128333
106245,true,1,True,True,False,8:0-18:0,8:0-18:0,8:0-18:0,8:0-18:0,8:0-18:0,...,0,0,A short while back I stopped in for a chicken ...,2017-12-12 15:06:09,2017,15:06:09,12,12,short back stopped chicken sandwich two actual...,-0.115152
194808,True,2.0,None,None,NaN,9:0-21:0,9:0-21:0,9:0-21:0,9:0-21:0,9:0-21:0,...,0,1,It's a typical city book store--a little bit s...,2014-01-28 15:59:40,2014,15:59:40,1,28,typical city book storea little bit small litt...,0.109405
246684,True,2.0,True,True,True,8:0-22:0,8:0-22:0,8:0-22:0,8:0-16:0,8:0-22:0,...,0,0,It wa ok. We had the opera cake and it wa all ...,2018-02-24 01:55:11,2018,01:55:11,2,24,wa ok opera cake wa right actually better day ...,0.375000
344456,True,1.0,True,True,NaN,5:0-20:30,5:0-20:30,5:0-20:30,5:0-20:30,5:0-20:30,...,0,0,This Dunkin Donuts get three star for it locat...,2014-03-14 14:27:16,2014,14:27:16,3,14,dunkin donuts get three star location hours tr...,0.221635
589830,True,2.0,True,True,True,NaN,NaN,9:0-14:0,9:0-14:0,9:0-14:0,...,0,0,Pancakes like undercooked bricks. Flavorless a...,2018-11-24 18:28:22,2018,18:28:22,11,24,pancakes like undercooked bricks flavorless av...,0.375000
660814,True,2.0,True,True,NaN,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,...,0,0,Love the bubble tea and friendly staff and coo...,2016-02-08 03:12:07,2016,03:12:07,2,8,love bubble tea friendly staff cool environmen...,0.245000
736954,False,2.0,True,False,NaN,NaN,11:0-17:0,11:0-17:0,11:0-17:0,11:0-18:0,...,1,0,I lived two block from here for 9 month before...,2012-09-06 19:53:12,2012,19:53:12,9,6,lived two block month finally trying delicious...,0.199872
999108,True,1.0,True,True,NaN,NaN,NaN,7:0-15:0,7:0-15:0,7:0-15:0,...,0,1,Perfect neighborhood cafe! They have delicious...,2013-08-17 23:39:34,2013,23:39:34,8,17,perfect neighborhood cafe delicious coffee app...,0.530303


In [25]:
res_list = list(second_target_df['name'].value_counts().index)

In [26]:
len(res_list) 

14

In [27]:
# 중복되는 아이디별 리뷰 텍스트에 대해 하나의 아이디에 대한 전체 리뷰 텍스트로 합침 
reviews = second_target_df[['name','text_clear']].groupby('name').sum()

In [28]:
reviews

,text_clear
name,
almaz cafe,weekend made first trip norristown year lived ...
b2,ive couple time time felt like wa bothering em...
barnes & noble booksellers,typical city book storea little bit small litt...
black & brew,perfect neighborhood cafe delicious coffee app...
cosi,going cosi terrible result since wa teenager k...
creme brulee bistro & cafe,wa ok opera cake wa right actually better day ...
dantè,reason place empty sunday prime brunch time hu...
dunkin',dunkin donuts get three star location hours tr...
federal donuts,short back stopped chicken sandwich two actual...


## tf-idf

In [29]:
import string
import re
import nltk

i = nltk.corpus.stopwords.words('english')
j = list(string.punctuation)
stopwords = set(i).union(j)

def preprocess(x):
    x = re.sub('[^a-z\s]', '', x.lower())                  # get rid of noise
    x = [w for w in x.split() if w not in set(stopwords)]  # remove stopwords
    return ' '.join(x) # then join the text again

texts = []

for index, row in reviews.iterrows():
    texts.append(preprocess(row['text_clear']))


In [30]:
%%time
# Estimated time: 29.8 s
from sklearn.feature_extraction.text import TfidfVectorizer

# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
# ngram은 단어의 묶음을 의미함. 이 단어 묶음의 범위를 설정하는 것이 ngram_range
vectorizer = TfidfVectorizer(ngram_range=(1,3)) # 단어의 묶음을 1개부터 3개까지 하라는 의미 ( => good과 같은 1개, very good과 같은 두개 그리고 세개짜리 모두 존재할 수 있다.)

# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(texts)

Wall time: 36.8 ms


In [31]:
vectors.shape

(14, 5377)

In [32]:
tfidv_df = pd.DataFrame(vectors.toarray(), columns = sorted(vectorizer.vocabulary_))
tfidv_df.index = reviews.index
tfidv_df

,able,able anything,able anything whats,absolutely,absolutely coming,absolutely coming back,absolutely regret,absolutely regret leaving,accept,accept level,...,yuengling,yuengling wa,yuengling wa miller,yum,yum although,yum although food,yup,yup three,yup three page,zahavs
name,,,,,,,,,,,,,,,,,,,,,
almaz cafe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.070595,0.070595,0.070595,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
b2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
barnes & noble booksellers,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
black & brew,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
cosi,0.035901,0.035901,0.035901,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
creme brulee bistro & cafe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
dantè,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.02287,0.02287,0.02287,0.000000
dunkin',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
federal donuts,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.057622


In [33]:
from sklearn.metrics.pairwise import cosine_similarity
tf_sim = cosine_similarity(vectors, vectors)

In [34]:
tf_sim.shape

(14, 14)

In [35]:
tfidf_sim_df = pd.DataFrame(data = tf_sim, index = tfidv_df.index, columns = tfidv_df.index)
tfidf_sim_df.head()

name,almaz cafe,b2,barnes & noble booksellers,black & brew,cosi,creme brulee bistro & cafe,dantè,dunkin',federal donuts,fuel,luna café,mr joe's cafe,plenty café,tea do
name,,,,,,,,,,,,,,
almaz cafe,1.000000,0.022068,0.011077,0.042298,0.037124,0.014621,0.051352,0.019101,0.018323,0.010992,0.000000,0.017395,0.035838,0.000000
b2,0.022068,1.000000,0.024707,0.109106,0.043790,0.026270,0.056935,0.034988,0.034938,0.010517,0.015030,0.015601,0.059247,0.020760
barnes & noble booksellers,0.011077,0.024707,1.000000,0.017607,0.006285,0.005354,0.012092,0.009483,0.000000,0.000000,0.000000,0.015734,0.004632,0.000000
black & brew,0.042298,0.109106,0.017607,1.000000,0.137790,0.040087,0.145590,0.044690,0.071784,0.078521,0.038455,0.041086,0.082137,0.026908
cosi,0.037124,0.043790,0.006285,0.137790,1.000000,0.043021,0.146869,0.027781,0.075873,0.027438,0.016105,0.018017,0.068954,0.003369


In [36]:
# 추천대상인 유저의 리뷰 데이터 가져오기 (평점 기분 정렬)
# 이후 가장 높은 평점을 가진 레스토랑에 대해서만 데이터 가져오기  (-> 이후 해당 레스토랑 기준 tf-idf 유사도 생성)
recomm_user_df = target_cat_df[target_cat_df['user_id']==recomm_user][['name','stars_y']].sort_values('stars_y', ascending=False)
tf_rest = []
for r in recomm_user_df.iterrows():
    tf_rest.append(r[1][0])
    break
tf_rest

['black & brew']

In [37]:
# recomm 대상 유저의 가장 높은 평점 레스토랑 데이터에 대해서 tf-idf 유사도가 가장 높은 순으로 레스토랑 정보 가져오기 
for t in tf_rest:
    _df = tfidf_sim_df[t].sort_values(ascending=False)[0:11]
    sim = []
    for d in _df:
        sim.append(d)
    phase2_df = pd.DataFrame({'name':_df.index, 'sim':sim})
    break

In [38]:
phase2_df

,name,sim
0,black & brew,1.000000
1,dantè,0.145590
2,cosi,0.137790
3,b2,0.109106
4,plenty café,0.082137
5,fuel,0.078521
6,federal donuts,0.071784
7,dunkin',0.044690
8,almaz cafe,0.042298
9,mr joe's cafe,0.041086


In [39]:
phase2_res = list(phase2_df.name)

## sentiment polarity

In [40]:
reviews.text_clear

name
almaz cafe                    weekend made first trip norristown year lived ...
b2                            ive couple time time felt like wa bothering em...
barnes & noble booksellers    typical city book storea little bit small litt...
black & brew                  perfect neighborhood cafe delicious coffee app...
cosi                          going cosi terrible result since wa teenager k...
creme brulee bistro & cafe    wa ok opera cake wa right actually better day ...
dantè                         reason place empty sunday prime brunch time hu...
dunkin'                       dunkin donuts get three star location hours tr...
federal donuts                short back stopped chicken sandwich two actual...
fuel                          ordered turkey burger special man wa delicious...
luna café                     pancakes like undercooked bricks flavorless av...
mr joe's cafe                 lived two block month finally trying delicious...
plenty café                   last 

In [41]:
from textblob import TextBlob
def sentiment(x):
    sentiment = TextBlob(x)
    return sentiment.sentiment.polarity

# sentiment(정서, 감정) polarity(극성) = 문장의 부정, 긍정의 정도에 따라 -1과 1사이의 값으로 표현
reviews['senti_polarity'] = reviews['text_clear'].apply(sentiment)
reviews =  reviews.reset_index()
reviews

,name,text_clear,senti_polarity
0,almaz cafe,weekend made first trip norristown year lived ...,0.128333
1,b2,ive couple time time felt like wa bothering em...,0.090131
2,barnes & noble booksellers,typical city book storea little bit small litt...,0.109405
3,black & brew,perfect neighborhood cafe delicious coffee app...,0.320701
4,cosi,going cosi terrible result since wa teenager k...,0.038127
5,creme brulee bistro & cafe,wa ok opera cake wa right actually better day ...,0.375000
6,dantè,reason place empty sunday prime brunch time hu...,-0.027806
7,dunkin',dunkin donuts get three star location hours tr...,0.221635
8,federal donuts,short back stopped chicken sandwich two actual...,-0.067210
9,fuel,ordered turkey burger special man wa delicious...,0.508163


In [42]:
phase2_senti = reviews[reviews['name'].isin(phase2_res)][['name','senti_polarity']]
phase2_senti

,name,senti_polarity
0,almaz cafe,0.128333
1,b2,0.090131
3,black & brew,0.320701
4,cosi,0.038127
5,creme brulee bistro & cafe,0.375000
6,dantè,-0.027806
7,dunkin',0.221635
8,federal donuts,-0.067210
9,fuel,0.508163
11,mr joe's cafe,0.199872


In [43]:
phase2_senti['senti_polarity'] = phase2_senti['senti_polarity'].round(decimals=3)
phase2_senti

,name,senti_polarity
0,almaz cafe,0.128
1,b2,0.090
3,black & brew,0.321
4,cosi,0.038
5,creme brulee bistro & cafe,0.375
6,dantè,-0.028
7,dunkin',0.222
8,federal donuts,-0.067
9,fuel,0.508
11,mr joe's cafe,0.200


In [44]:
phase2_df = pd.merge(phase2_df,phase2_senti, on='name')

In [45]:
phase2_df = phase2_df.sort_values('senti_polarity', ascending=False)
phase2_df

,name,sim,senti_polarity
5,fuel,0.078521,0.508
10,creme brulee bistro & cafe,0.040087,0.375
0,black & brew,1.000000,0.321
7,dunkin',0.044690,0.222
9,mr joe's cafe,0.041086,0.200
8,almaz cafe,0.042298,0.128
3,b2,0.109106,0.090
4,plenty café,0.082137,0.063
2,cosi,0.137790,0.038
1,dantè,0.145590,-0.028


# 최종 Top3 추천

In [128]:
phase1_df

,name,stars_y
1122397,reading terminal market,5
766510,lucha cartel,5
833596,profi's crêperie,5
957131,barra rossa ristorante,5
750882,triangle tavern,5
740861,the m room,5
737484,george's market at dreshertown,5
581553,wrap shack,5
999227,black & brew,5
1008994,federal beer distributor,5


In [129]:
phase2_df

,name,sim,senti_polarity
5,fuel,0.078521,0.508
10,creme brulee bistro & cafe,0.040087,0.375
0,black & brew,1.000000,0.321
7,dunkin',0.044690,0.222
9,mr joe's cafe,0.041086,0.200
8,almaz cafe,0.042298,0.128
3,b2,0.109106,0.090
4,plenty café,0.082137,0.063
2,cosi,0.137790,0.038
1,dantè,0.145590,-0.028


In [135]:
# phase 1,2 만 고려한 경우 
merge_phase = pd.merge(phase1_df,phase2_df,how='outer',on='name').sort_values(['sim', 'senti_polarity'],ascending=False)
merge_phase

,name,stars_y,sim,senti_polarity
8,black & brew,5.0,1.000000,0.321
23,dantè,NaN,0.145590,-0.028
22,cosi,NaN,0.137790,0.038
20,b2,NaN,0.109106,0.090
21,plenty café,NaN,0.082137,0.063
15,fuel,NaN,0.078521,0.508
24,federal donuts,NaN,0.071784,-0.067
17,dunkin',NaN,0.044690,0.222
19,almaz cafe,NaN,0.042298,0.128
18,mr joe's cafe,NaN,0.041086,0.200


In [136]:
merge_phase.sort_values('senti_polarity', ascending=False)

,name,stars_y,sim,senti_polarity
15,fuel,NaN,0.078521,0.508
16,creme brulee bistro & cafe,NaN,0.040087,0.375
8,black & brew,5.0,1.000000,0.321
17,dunkin',NaN,0.044690,0.222
18,mr joe's cafe,NaN,0.041086,0.200
19,almaz cafe,NaN,0.042298,0.128
20,b2,NaN,0.109106,0.090
21,plenty café,NaN,0.082137,0.063
22,cosi,NaN,0.137790,0.038
23,dantè,NaN,0.145590,-0.028


In [137]:
double_res_df = pd.read_csv('./double_res.csv', index_col=0)
double_res_df

,name,stars_y
0,aldi,5.0
1,musashi exotic japanese cuisine,5.0
2,sang kee noodle house,5.0
3,anastasi seafood,5.0
4,frieda,5.0
...,...,...
9623,m restaurant,4.5
9635,leo's steak shop,4.5
9637,wishbone,4.5
9639,suvege,5.0


In [138]:
double_res = list(double_res_df['name'])

In [139]:
# 사용자가 여러번 방문한 레스토랑에 대해
weight_phase_df = merge_phase[merge_phase['name'].isin(double_res)]

In [140]:
weight_phase_df

,name,stars_y,sim,senti_polarity
8,black & brew,5.0,1.000000,0.321
22,cosi,NaN,0.137790,0.038
20,b2,NaN,0.109106,0.090
21,plenty café,NaN,0.082137,0.063
15,fuel,NaN,0.078521,0.508
24,federal donuts,NaN,0.071784,-0.067
17,dunkin',NaN,0.044690,0.222
19,almaz cafe,NaN,0.042298,0.128
16,creme brulee bistro & cafe,NaN,0.040087,0.375
0,reading terminal market,5.0,NaN,NaN


In [141]:
weighted_phase = weighted_phase.sort_values(['senti_polarity'], ascending=False)
weighted_phase


,name,stars_y,sim,senti_polarity
117,fuel,2.0,0.078521,0.508
131,creme brulee bistro & cafe,0.0,0.040087,0.375
73,black & brew,4.0,1.000000,0.321
132,dunkin',0.0,0.044690,0.222
134,almaz cafe,0.0,0.042298,0.128
...,...,...,...,...
126,beiler's bakery,2.0,0.000000,0.000
127,indian restaurant,1.0,0.000000,0.000
129,the original tony lukes,1.0,0.000000,0.000
130,the sweet taco,1.0,0.000000,0.000
